<a href="https://colab.research.google.com/github/matthewchung74/inference_nbs/blob/main/huggingface/question_answering_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pathlib import Path

def create_requirements_file():
    if not Path("requirements.txt").exists():
        requirements = ["transformers"]
        with open('requirements.txt', 'w') as filehandle:
            for listitem in requirements:
                filehandle.write('%s\n' % listitem)
    
create_requirements_file()

In [2]:
!pip install -r requirements.txt

     |████████████████████████████████| 2.1MB 11.4MB/s 
     |████████████████████████████████| 901kB 52.2MB/s 
     |████████████████████████████████| 3.3MB 53.8MB/s 


In [3]:
!wget https://ml-inference.s3-us-west-2.amazonaws.com/hugging_face_question_answering_training.zip

--2021-04-27 00:14:18--  https://ml-inference.s3-us-west-2.amazonaws.com/hugging_face_question_answering_training.zip
Resolving ml-inference.s3-us-west-2.amazonaws.com (ml-inference.s3-us-west-2.amazonaws.com)... 52.218.232.129
Connecting to ml-inference.s3-us-west-2.amazonaws.com (ml-inference.s3-us-west-2.amazonaws.com)|52.218.232.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244710271 (233M) [application/zip]
Saving to: ‘hugging_face_question_answering_training.zip’

hugging_face_questi 100%[===================>] 233.37M  18.6MB/s    in 14s     

2021-04-27 00:14:33 (16.9 MB/s) - ‘hugging_face_question_answering_training.zip’ saved [244710271/244710271]



In [4]:
!unzip hugging_face_question_answering_training.zip

Archive:  hugging_face_question_answering_training.zip
   creating: model/
  inflating: model/config.json       
   creating: model/tokenizer/
  inflating: model/tokenizer/vocab.txt  
  inflating: model/pytorch_model.bin  


In [5]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, AutoConfig
import torch
from pathlib import Path

model_path = Path("model")

tokenizer_new = AutoTokenizer.from_pretrained(str(model_path/"tokenizer"), config=AutoConfig.from_pretrained(model_path))
model_new = AutoModelForQuestionAnswering.from_pretrained(model_path)
model_new.cpu();

In [6]:
!pip install -q git+https://github.com/matthewchung74/inference_params.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 61kB 6.8MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [7]:
from inference_params.inference_params import inference_test, FieldType, inference_predict

# text input will be the label for the app input
input = {"question": FieldType.Text, "context": FieldType.Text}
# result will be the label for the app output
output = {"result": FieldType.Text}

@inference_predict(input=input, output=output)
def predict(inputs):
    question = inputs["question"]
    context = inputs["context"]
    inputs = tokenizer_new.encode_plus(question, context,add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    output = model_new(**inputs)
    answer_start_scores = output.start_logits
    answer_end_scores = output.end_logits

    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer_new.convert_tokens_to_string(tokenizer_new.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    return {"result": answer}

In [9]:
from inference_params.inference_params import in_colab

if in_colab():
    context = r"""
    🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
    architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
    Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
    TensorFlow 2.0 and PyTorch.
    """

    params = {"question":"How many pretrained models are available in Transformers?",
            "context":context}

    inference_test(predict_func=predict, params=params)

Wrote results to result.json duration: 0.177052 seconds
Please take a look and verify the results
